# 대도시권 광역교통 빅데이터 분석 시각화 및 활용 아이디어 경진대회 [링크](https://dacon.io/competitions/official/236009/overview/description)

### 주제: 서울, 경기, 인천권 광역교통 분석을 통해 대중교통 노선이 부족한 지역을 찾아 노선 신설 또는 연장 제안

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import openpyxl

from matplotlib import rc
rc("font", family = "Malgun Gothic")

get_ipython().run_line_magic("matplotlib", "inline")

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.keys import Keys

import time
from tqdm.notebook import tqdm

from glob import glob
import warnings
warnings.filterwarnings('ignore')

## 제공 데이터 설명
```
<지역>
수도권: 서울, 인천, 경기도
부울권: 부산, 울산, 양산, 김해, 창원, 경주
대구권: 대구, 구미, 경산, 영천, 군위, 청도, 고령, 성주, 칠곡, 창녕
광주권: 광주, 나주, 담양, 화순, 함평, 장성
대전권: 대전, 세종, 공주, 논산, 계룡, 금산, 청주, 보은, 옥천
<시간>
2019-2021년 3월 중 평일 1일
```

## 내용
```
Stop : 대중교통 정차지 정보
Route : 대중교통 노선 기초정보
RouteStop : 대중교통 노선별 경유정차지 정보
StopTime : 대중교통 노선별, 운행회차별 정차순번 및 도착/출발시각 정보
```

In [ ]:
Stop_bus_2019 = pd.read_excel("./2019.03/201903_1_버스_1_BaseInfo.xlsx", sheet_name=0)
Route_bus_2019 = pd.read_excel("./2019.03/201903_1_버스_1_BaseInfo.xlsx", sheet_name=1)
RouteStop_bus_2019 = pd.read_excel("./2019.03/201903_1_버스_1_BaseInfo.xlsx", sheet_name=2)
StopTime_bus_2019 = pd.read_csv("./2019.03/201903_1_버스_2_StopTime.txt")

Stop_bus_2020 = pd.read_excel("./2020.03/202003_1_버스_1_BaseInfo.xlsx", sheet_name=0)
Route_bus_2020 = pd.read_excel("./2020.03/202003_1_버스_1_BaseInfo.xlsx", sheet_name=1)
RouteStop_bus_2020 = pd.read_excel("./2020.03/202003_1_버스_1_BaseInfo.xlsx", sheet_name=2)
StopTime_bus_2020 = pd.read_csv("./2020.03/202003_1_버스_2_StopTime.txt")

Stop_bus_2021 = pd.read_excel("./2021.03/202103_1_버스_1_BaseInfo.xlsx", sheet_name=0)
Route_bus_2021 = pd.read_excel("./2021.03/202103_1_버스_1_BaseInfo.xlsx", sheet_name=1)
RouteStop_bus_2021 = pd.read_excel("./2021.03/202103_1_버스_1_BaseInfo.xlsx", sheet_name=2)
StopTime_bus_2021 = pd.read_csv("./2021.03/202103_1_버스_2_StopTime.txt")

In [ ]:
Stop_rail_2019 = pd.read_excel("./2019.03/201903_2_도시철도.xlsx", sheet_name=0)
Xfer_rail_2019 = pd.read_excel("./2019.03/201903_2_도시철도.xlsx", sheet_name=1)
Route_rail_2019 = pd.read_excel("./2019.03/201903_2_도시철도.xlsx", sheet_name=2)
RouteStop_rail_2019 = pd.read_excel("./2019.03/201903_2_도시철도.xlsx", sheet_name=3)
StopTime_rail_2019 = pd.read_excel("./2019.03/201903_2_도시철도.xlsx", sheet_name=4)

Stop_rail_2020 = pd.read_excel("./2020.03/202003_2_도시철도.xlsx", sheet_name=0)
Xfer_rail_2020 = pd.read_excel("./2020.03/202003_2_도시철도.xlsx", sheet_name=1)
Route_rail_2020 = pd.read_excel("./2020.03/202003_2_도시철도.xlsx", sheet_name=2)
RouteStop_rail_2020 = pd.read_excel("./2020.03/202003_2_도시철도.xlsx", sheet_name=3)
StopTime_rail_2020 = pd.read_excel("./2020.03/202003_2_도시철도.xlsx", sheet_name=4)

Stop_rail_2021 = pd.read_excel("./2021.03/202103_2_도시철도.xlsx", sheet_name=0)
Xfer_rail_2021 = pd.read_excel("./2021.03/202103_2_도시철도.xlsx", sheet_name=1)
Route_rail_2021 = pd.read_excel("./2021.03/202103_2_도시철도.xlsx", sheet_name=2)
RouteStop_rail_2021 = pd.read_excel("./2021.03/202103_2_도시철도.xlsx", sheet_name=3)
StopTime_rail_2021 = pd.read_excel("./2021.03/202103_2_도시철도.xlsx", sheet_name=4)

---
# 1. 통행량 데이터 수집
#### [교통카드 빅데이터 종합정보시스템](https://stcis.go.kr/pivotIndi/wpsPivotIndicator.do?siteGb=P&indiClss=IC04) 페이지에서 2019-2021년 3월 평일 중 하루의 통행량 데이터 수집.
#### 단위는 [시군구] 로 설정.

In [6]:
url = "https://stcis.go.kr/pivotIndi/wpsPivotIndicator.do?siteGb=P&indiClss=IC04"
chrome_options = webdriver.ChromeOptions()
driver = webdriver.Chrome(service=Service('C:/Users/gktna/Documents/study/driver/chromedriver.exe'), options=chrome_options)
driver.get(url)

In [23]:
driver.find_element(By.XPATH, '//*[@id="divRdoODAear"]/li[2]')
driver.find_element(By.XPATH, '//*[@id="searchStgptZoneSd"]').send_keys("서울특별시")
ss_list = driver.find_element(By.XPATH, '//*[@id="searchStgptZoneSgg"]').text.split('\n')[1:]
ss_list

['강남구',
 '강동구',
 '강북구',
 '강서구',
 '관악구',
 '광진구',
 '구로구',
 '금천구',
 '노원구',
 '도봉구',
 '동대문구',
 '동작구',
 '마포구',
 '서대문구',
 '서초구',
 '성동구',
 '성북구',
 '송파구',
 '양천구',
 '영등포구',
 '용산구',
 '은평구',
 '종로구',
 '중구',
 '중랑구']

In [27]:
driver.find_element(By.XPATH, '//*[@id="searchStgptZoneSd"]').send_keys("경기도")
gg_list = driver.find_element(By.XPATH, '//*[@id="searchStgptZoneSgg"]').text.split('\n')[1:]
gg_list

['가평군',
 '고양시',
 '과천시',
 '광명시',
 '광주시',
 '구리시',
 '군포시',
 '김포시',
 '남양주시',
 '동두천시',
 '부천시',
 '성남시',
 '수원시',
 '시흥시',
 '안산시',
 '안성시',
 '안양시',
 '양주시',
 '양평군',
 '여주시',
 '연천군',
 '오산시',
 '용인시',
 '의왕시',
 '의정부시',
 '이천시',
 '파주시',
 '평택시',
 '포천시',
 '하남시',
 '화성시']

In [26]:
driver.find_element(By.XPATH, '//*[@id="searchStgptZoneSd"]').send_keys("인천광역시")
ii_list = driver.find_element(By.XPATH, '//*[@id="searchStgptZoneSgg"]').text.split('\n')[1:]
ii_list

['강화군', '계양구', '남동구', '동구', '미추홀구', '부평구', '서구', '연수구', '옹진군', '중구']

In [28]:
si_list = ['서울특별시', '인천광역시', '경기도']
gu_list = [ss_list, ii_list, gg_list]

In [32]:
for i in tqdm(range(0, 3)):
    driver.find_element(By.XPATH, '//*[@id="divRdoDate"]/li[3]').click() # 기간선택-일
    driver.find_element(By.XPATH, '//*[@id="date3"]/li[1]/img').click() # 달력그림 (위)
    driver.find_element(By.XPATH, '//*[@id="ui-datepicker-div"]/div/div/select[1]/option[1]').click() # 연도 선택
    driver.find_element(By.XPATH, '//*[@id="ui-datepicker-div"]/div/div/select[2]/option[3]').click() # 3월
    driver.find_element(By.XPATH, '//*[@id="ui-datepicker-div"]/table/tbody/tr[3]/td[4]').click() # 평일 날짜 선택

    driver.find_element(By.XPATH, '//*[@id="date3"]/li[2]/img').click() # 달력그림 (아래)
    driver.find_element(By.XPATH, '//*[@id="ui-datepicker-div"]/div/div/select[1]/option[1]').click() # 연도 선택 
    driver.find_element(By.XPATH, '//*[@id="ui-datepicker-div"]/div/div/select[2]/option[1]').click() # 3월
    driver.find_element(By.XPATH, '//*[@id="ui-datepicker-div"]/table/tbody/tr[3]/td[4]').click() # 평일 날짜 선택

    for k in range(len(gu_list[i])):

        driver.find_element(By.XPATH, '//*[@id="divRdoODAear"]/li[2]').click() # 출발지 선택 - 시군구

        driver.find_element(By.XPATH, '//*[@id="searchStgptZoneSd"]').send_keys(si_list[i]) # 시도 입력

        driver.find_element(By.XPATH, '//*[@id="searchStgptZoneSgg"]').send_keys(gu_list[i][k]) # 시군구 입력

        driver.find_element(By.XPATH, '//*[@id="OD_end_box"]/div[1]/ul[2]/li/label[2]').click()  # 도착지 선택 - 시군구전체선택 클릭

        driver.find_element(By.XPATH, '//*[@id="btnSearch"]/button').click() # 결과보기 버튼

        time.sleep(5)

        try:
            driver.find_element(By.XPATH, '//*[@id="rgrstyReportResult"]/h2/p/span[1]').click()  #다운로드 버튼
        except:
            print( gu_list[i][k], ' 다운로드 실패')
            



        time.sleep(5)

        driver.find_element(By.XPATH, '//*[@id="tab1"]/div[2]/div[3]/button').click() # 뒤로가기 버튼

        time.sleep(1) 

  0%|          | 0/3 [00:00<?, ?it/s]

In [33]:
driver.close()
driver.quit()

---
# 2. 전철역, 버스정류장 주소 수집
#### 카카오 맵 이용
#### 위도, 경도 값으로 검색
#### 정류장 이름, 시/도, 시/군/구, 동/읍/면, 위도, 경도 값 저장

In [ ]:
url = "https://map.kakao.com/"
chrome_options = webdriver.ChromeOptions()
driver = webdriver.Chrome(service=Service('C:/Users/gktna/Documents/study/driver/chromedriver.exe'), options=chrome_options)
driver.get(url)

### (1) 전철역

#### 2019년

In [ ]:
Stop_rail_2019['Lati'] = Stop_rail_2019['Lati'].astype(str)
Stop_rail_2019['Long'] = Stop_rail_2019['Long'].astype(str)
Stop_rail_2019.info()

In [ ]:
list_1 = []
list_2 = []
list_3 = []
list_4 = []
list_5 = []
list_6 = []
list_7 = []


for idx, row in tqdm(Stop_rail_2019.iterrows()):
    t = str(row['Lati'] + ", " + row['Long'])
    driver.find_element(By.XPATH, '//*[@id="search.keyword.query"]').send_keys(t)
    driver.find_element(By.XPATH,'//*[@id="search.keyword.submit"]').send_keys(Keys.RETURN)
    time.sleep(2)
    
    try:
        s = driver.find_element(By.XPATH, '//*[@id="localInfo.map"]/div[1]').text
        g = driver.find_element(By.XPATH, '//*[@id="localInfo.map"]/div[2]').text
        d = driver.find_element(By.XPATH, '//*[@id="localInfo.map"]/div[3]').text
     
    except:
        s = "1"
        g = "1"
        d = "1"
        
    list_1.append(s) 
    list_2.append(g) 
    list_3.append(d) 
    list_4.append(row['Lati'])
    list_5.append(row['Long'])
    list_6.append(row['Stop_ID'])
    list_7.append(row['Stop_NM'])
     
        
    driver.find_element(By.XPATH,'//*[@id="search.keyword.query"]').clear()

In [ ]:
stop_rail_2019_df = pd.DataFrame(list(zip(list_6, list_7list_1, list_2 , list_3 , list_4, list_5)), 
                            columns=["Stop_ID", "Stop_NM", '시/도',"시/군/구", '동', 'Lati', 'Long'])
stop_rail_2019_df

#### 2020년

In [ ]:
Stop_rail_2020['Lati'] = Stop_rail_2020['Lati'].astype(str)
Stop_rail_2020['Long'] = Stop_rail_2020['Long'].astype(str)
Stop_rail_2020.info()

In [ ]:
list_1 = []
list_2 = []
list_3 = []
list_4 = []
list_5 = []
list_6 = []
list_7 = []


for idx, row in tqdm(Stop_rail_2020.iterrows()):
    t = str(row['Lati'] + ", " + row['Long'])
    driver.find_element(By.XPATH, '//*[@id="search.keyword.query"]').send_keys(t)
    driver.find_element(By.XPATH,'//*[@id="search.keyword.submit"]').send_keys(Keys.RETURN)
    time.sleep(2)
    
    try:
        s = driver.find_element(By.XPATH, '//*[@id="localInfo.map"]/div[1]').text
        g = driver.find_element(By.XPATH, '//*[@id="localInfo.map"]/div[2]').text
        d = driver.find_element(By.XPATH, '//*[@id="localInfo.map"]/div[3]').text
     
    except:
        s = "1"
        g = "1"
        d = "1"
        
    list_1.append(s) 
    list_2.append(g) 
    list_3.append(d) 
    list_4.append(row['Lati'])
    list_5.append(row['Long'])
    list_6.append(row['Stop_ID'])
    list_7.append(row['Stop_NM'])
     
        
    driver.find_element(By.XPATH,'//*[@id="search.keyword.query"]').clear()

In [ ]:
stop_rail_2020_df = pd.DataFrame(list(zip(list_6, list_7list_1, list_2 , list_3 , list_4, list_5)), 
                            columns=["Stop_ID", "Stop_NM", '시/도',"시/군/구", '동', 'Lati', 'Long'])
stop_rail_2020_df

#### 2021년

In [44]:
Stop_rail_2021['Lati'] = Stop_rail_2021['Lati'].astype(str)
Stop_rail_2021['Long'] = Stop_rail_2021['Long'].astype(str)
Stop_rail_2021.info()

In [ ]:
list_1 = []
list_2 = []
list_3 = []
list_4 = []
list_5 = []
list_6 = []
list_7 = []


for idx, row in tqdm(Stop_rail_2021.iterrows()):
    t = str(row['Lati'] + ", " + row['Long'])
    driver.find_element(By.XPATH, '//*[@id="search.keyword.query"]').send_keys(t)
    driver.find_element(By.XPATH,'//*[@id="search.keyword.submit"]').send_keys(Keys.RETURN)
    time.sleep(2)
    
    try:
        s = driver.find_element(By.XPATH, '//*[@id="localInfo.map"]/div[1]').text
        g = driver.find_element(By.XPATH, '//*[@id="localInfo.map"]/div[2]').text
        d = driver.find_element(By.XPATH, '//*[@id="localInfo.map"]/div[3]').text
     
    except:
        s = "1"
        g = "1"
        d = "1"
        
    list_1.append(s) 
    list_2.append(g) 
    list_3.append(d) 
    list_4.append(row['Lati'])
    list_5.append(row['Long'])
    list_6.append(row['Stop_ID'])
    list_7.append(row['Stop_NM'])
     
        
    driver.find_element(By.XPATH,'//*[@id="search.keyword.query"]').clear()

In [43]:
stop_rail_2020_df = pd.DataFrame(list(zip(list_6, list_7list_1, list_2 , list_3 , list_4, list_5)), 
                            columns=["Stop_ID", "Stop_NM", '시/도',"시/군/구", '동', 'Lati', 'Long'])
stop_rail_2020_df

,시/도,시/군/구,동,Lati,Long
0,서울특별시,중구,회현동,37.55587,126.9721
1,서울특별시,중구,소공동,37.56571,126.97712
2,서울특별시,종로구,종로1.2.3.4가동,37.57016,126.98292
3,서울특별시,종로구,종로1.2.3.4가동,37.57042,126.99211
4,서울특별시,종로구,종로5.6가동,37.57093,127.00185
...,...,...,...,...,...
1074,경기도,하남시,덕풍3동,37.55215,127.203927
1075,경기도,하남시,덕풍3동,37.541913,127.206404
1076,경기도,하남시,천현동,37.539746,127.223271
1077,인천광역시,연수구,송도2동,37.406767,126.626094


### (2) 버스 정류장

#### 2019년

In [ ]:
Stop_bus_2019['Lati'] = Stop_bus_2019['Lati'].astype(str)
Stop_bus_2019['Long'] = Stop_bus_2019['Long'].astype(str)
Stop_bus_2019.info()

In [ ]:
list_1 = []
list_2 = []
list_3 = []
list_4 = []
list_5 = []
list_6 = []
list_7 = []


for idx, row in tqdm(Stop_rail_2019.iterrows()):
    t = str(row['Lati'] + ", " + row['Long'])
    driver.find_element(By.XPATH, '//*[@id="search.keyword.query"]').send_keys(t)
    driver.find_element(By.XPATH,'//*[@id="search.keyword.submit"]').send_keys(Keys.RETURN)
    time.sleep(2)
    
    try:
        s = driver.find_element(By.XPATH, '//*[@id="localInfo.map"]/div[1]').text
        g = driver.find_element(By.XPATH, '//*[@id="localInfo.map"]/div[2]').text
        d = driver.find_element(By.XPATH, '//*[@id="localInfo.map"]/div[3]').text
     
    except:
        s = "1"
        g = "1"
        d = "1"
        
    list_1.append(s) 
    list_2.append(g) 
    list_3.append(d) 
    list_4.append(row['Lati'])
    list_5.append(row['Long'])
    list_6.append(row['Stop_ID'])
    list_7.append(row['Stop_NM'])
     
        
    driver.find_element(By.XPATH,'//*[@id="search.keyword.query"]').clear()

In [ ]:
stop_bus_2019_df = pd.DataFrame(list(zip(list_6, list_7list_1, list_2 , list_3 , list_4, list_5)), 
                            columns=["Stop_ID", "Stop_NM", '시/도',"시/군/구", '동', 'Lati', 'Long'])
stop_bus_2019_df

#### 2020년

In [ ]:
Stop_bus_2020['Lati'] = Stop_bus_2020['Lati'].astype(str)
Stop_bus_2020['Long'] = Stop_bus_2020['Long'].astype(str)
Stop_bus_2020.info()

In [ ]:
list_1 = []
list_2 = []
list_3 = []
list_4 = []
list_5 = []
list_6 = []
list_7 = []

for idx, row in tqdm(Stop_rail_2019.iterrows()):
    t = str(row['Lati'] + ", " + row['Long'])
    driver.find_element(By.XPATH, '//*[@id="search.keyword.query"]').send_keys(t)
    driver.find_element(By.XPATH,'//*[@id="search.keyword.submit"]').send_keys(Keys.RETURN)
    time.sleep(2)
    
    try:
        s = driver.find_element(By.XPATH, '//*[@id="localInfo.map"]/div[1]').text
        g = driver.find_element(By.XPATH, '//*[@id="localInfo.map"]/div[2]').text
        d = driver.find_element(By.XPATH, '//*[@id="localInfo.map"]/div[3]').text
     
    except:
        s = "1"
        g = "1"
        d = "1"
        
    list_1.append(s) 
    list_2.append(g) 
    list_3.append(d) 
    list_4.append(row['Lati'])
    list_5.append(row['Long'])
    list_6.append(row['Stop_ID'])
    list_7.append(row['Stop_NM'])
     
        
    driver.find_element(By.XPATH,'//*[@id="search.keyword.query"]').clear()

In [ ]:
stop_bus_2020_df = pd.DataFrame(list(zip(list_6, list_7list_1, list_2 , list_3 , list_4, list_5)), 
                            columns=["Stop_ID", "Stop_NM", '시/도',"시/군/구", '동', 'Lati', 'Long'])
stop_bus_2020_df

#### 2021년

In [ ]:
Stop_bus_2021['Lati'] = Stop_bus_2021['Lati'].astype(str)
Stop_bus_2021['Long'] = Stop_bus_2021['Long'].astype(str)
Stop_bus_2021.info()

In [ ]:
list_1 = []
list_2 = []
list_3 = []
list_4 = []
list_5 = []
list_6 = []
list_7 = []

for idx, row in tqdm(Stop_rail_2019.iterrows()):
    t = str(row['Lati'] + ", " + row['Long'])
    driver.find_element(By.XPATH, '//*[@id="search.keyword.query"]').send_keys(t)
    driver.find_element(By.XPATH,'//*[@id="search.keyword.submit"]').send_keys(Keys.RETURN)
    time.sleep(2)
    
    try:
        s = driver.find_element(By.XPATH, '//*[@id="localInfo.map"]/div[1]').text
        g = driver.find_element(By.XPATH, '//*[@id="localInfo.map"]/div[2]').text
        d = driver.find_element(By.XPATH, '//*[@id="localInfo.map"]/div[3]').text
     
    except:
        s = "1"
        g = "1"
        d = "1"
        
    list_1.append(s) 
    list_2.append(g) 
    list_3.append(d) 
    list_4.append(row['Lati'])
    list_5.append(row['Long'])
    list_6.append(row['Stop_ID'])
    list_7.append(row['Stop_NM'])
     
        
    driver.find_element(By.XPATH,'//*[@id="search.keyword.query"]').clear()

In [ ]:
stop_bus_2021_df = pd.DataFrame(list(zip(list_6, list_7list_1, list_2 , list_3 , list_4, list_5)), 
                            columns=["Stop_ID", "Stop_NM", '시/도',"시/군/구", '동', 'Lati', 'Long'])
stop_bus_2021_df

---
# 3. 통행량 데이터 전처리
## 1) 파일 불러오기
- glob 모듈 사용
- dataframe 형태로 변환
- 도착지가 서울, 경기, 인천인 데이터만 추출하여 저장

In [6]:
od_files_2019 = glob("./OD_2019_03/이용객*.xlsx")
od_files_2020 = glob("./OD_2020_03/이용객*.xlsx")
od_files_2021 = glob("./OD_2021_03/이용객*.xlsx")

tmp_raw = []

for file_name in od_files_2019:
    tmp = pd.read_excel(file_name, header=0)
    tmp_raw.append(tmp)
    
for file_name in od_files_2020:
    tmp = pd.read_excel(file_name, header=0)
    tmp_raw.append(tmp)
    
for file_name in od_files_2021:
    tmp = pd.read_excel(file_name, header=0)
    tmp_raw.append(tmp)

In [7]:
od_raw = pd.concat(tmp_raw)
od_df = pd.DataFrame({"일자":od_raw["일자"],
                        "시도(출발)":od_raw["시도(출발)"].str.strip(),
                        "시군구(출발)":od_raw["시군구(출발)"].str.strip(),
                        "시도(도착)":od_raw["시도(도착)"].str.strip(),
                        "시군구(도착)":od_raw["시군구(도착)"].str.strip(),
                         "통행량":od_raw["통행량"]})
od_df.head()

,일자,시도(출발),시군구(출발),시도(도착),시군구(도착),통행량
0,2019-03-13(수),서울특별시,강동구,강원도,춘천시,71
1,2019-03-13(수),서울특별시,강동구,경기도,가평군,28
2,2019-03-13(수),서울특별시,강동구,경기도,고양시,704
3,2019-03-13(수),서울특별시,강동구,경기도,과천시,218
4,2019-03-13(수),서울특별시,강동구,경기도,광명시,271


In [8]:
od_df.reset_index(inplace=True, drop=True)

for idx, row in tqdm(od_df.iterrows()):
    od_df.iloc[idx, 0] = row["일자"][:-6]
    
od_df.head()

0it [00:00, ?it/s]

,일자,시도(출발),시군구(출발),시도(도착),시군구(도착),통행량
0,2019-03,서울특별시,강동구,강원도,춘천시,71
1,2019-03,서울특별시,강동구,경기도,가평군,28
2,2019-03,서울특별시,강동구,경기도,고양시,704
3,2019-03,서울특별시,강동구,경기도,과천시,218
4,2019-03,서울특별시,강동구,경기도,광명시,271


In [9]:
od_df = od_df[od_df['시도(도착)'].isin(['서울특별시', '인천광역시', '경기도'])]
od_df.reset_index(drop=True, inplace=True)
od_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12411 entries, 0 to 12410
Data columns (total 6 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   일자       12411 non-null  object
 1   시도(출발)   12411 non-null  object
 2   시군구(출발)  12411 non-null  object
 3   시도(도착)   12411 non-null  object
 4   시군구(도착)  12411 non-null  object
 5   통행량      12411 non-null  int64 
dtypes: int64(1), object(5)
memory usage: 581.9+ KB


## 2) 피벗테이블 만들기
- 통행량은 쌍방.
- 시작, 도착 지점이 같으면 합쳐줘야한다

In [ ]:
order_list = ['경기도 가평군', '경기도 고양시', '경기도 과천시', '경기도 광명시', '경기도 광주시', '경기도 구리시',
       '경기도 군포시', '경기도 김포시', '경기도 남양주시', '경기도 동두천시', '경기도 부천시', '경기도 성남시 ',
       '경기도 수원시 ', '경기도 시흥시', '경기도 안산시 ', '경기도 안성시', '경기도 안양시 ', '경기도 양주시',
       '경기도 양평군', '경기도 여주시', '경기도 연천군', '경기도 오산시', '경기도 용인시 ', '경기도 의왕시',
       '경기도 의정부시', '경기도 이천시', '경기도 파주시', '경기도 평택시', '경기도 포천시', '경기도 하남시',
       '경기도 화성시', '서울특별시 강남구', '서울특별시 강동구', '서울특별시 강북구', '서울특별시 강서구',
       '서울특별시 관악구', '서울특별시 광진구', '서울특별시 구로구', '서울특별시 금천구', '서울특별시 노원구',
       '서울특별시 도봉구', '서울특별시 동대문구', '서울특별시 동작구', '서울특별시 마포구', '서울특별시 서대문구',
       '서울특별시 서초구', '서울특별시 성동구', '서울특별시 성북구', '서울특별시 송파구', '서울특별시 양천구',
       '서울특별시 영등포구', '서울특별시 용산구', '서울특별시 은평구', '서울특별시 종로구', '서울특별시 중구',
       '서울특별시 중랑구', '인천광역시 강화군', '인천광역시 계양구', '인천광역시 남동구', '인천광역시 동구',
       '인천광역시 미추홀구', '인천광역시 부평구', '인천광역시 서구', '인천광역시 연수구', '인천광역시 중구', 
              '인천광역시 옹진군']

o_li = [i.strip() for i in order_list]

In [ ]:
for i in range(2019,2022):
    st_df = pd.DataFrame(pd.pivot_table(data=od_df[od_df['일자'] == "{yr}-03".format(yr=i)], 
                                        index=["시도(출발)", "시군구(출발)"], 
                                        columns=["시도(도착)", "시군구(도착)"],
                                        values="통행량", aggfunc=np.sum, fill_value=0))

    s_tmp1 = [
        st_df.columns.get_level_values(0)[n] 
        +" "+ st_df.columns.get_level_values(1)[n]
        for n in range(0, len(st_df.columns.get_level_values(0)))]


    s_tmp2 = [
        st_df.index.get_level_values(0)[n] 
        +" "+ st_df.index.get_level_values(1)[n]
        for n in range(0, len(st_df.index.get_level_values(0)))]

    st_df.columns = s_tmp1
    st_df.index = s_tmp2


    ed_df = pd.DataFrame(pd.pivot_table(data=od_df[od_df['일자'] == "{yr}-03".format(yr=i)], 
                   index=["시도(도착)", "시군구(도착)"],
                   columns=["시도(출발)", "시군구(출발)"],             
                   values="통행량", aggfunc=np.sum, fill_value=0))


    e_tmp1 = [
        ed_df.columns.get_level_values(0)[n] 
        +" "+ ed_df.columns.get_level_values(1)[n]
        for n in range(0, len(ed_df.columns.get_level_values(0)))]


    e_tmp2 = [
        ed_df.index.get_level_values(0)[n] 
        +" "+ ed_df.index.get_level_values(1)[n]
        for n in range(0, len(ed_df.index.get_level_values(0)))]

    ed_df.columns = e_tmp1
    ed_df.index = e_tmp2

    st_df = st_df[o_li]
    st_df.reindex(o_li)

    ed_df = ed_df[o_li]
    ed_df.reindex(o_li)

    tmp_df = st_df + ed_df
    tmp_df.to_excel('./day/{y}년통행량.xlsx'.format(y=i), encoding='utf-8')

## 3) 피벗테이블의 인덱스, 컬럼명, 값 을 하나의 열로 갖는 DataFrame 만들기
- 시각화, merge 등을 위해 필요한 형태로 만든다.
- 모든 연도를 다 해준다,

In [ ]:
pass_2021 = pd.read_excel('./2021년통행량.', index_col=0)
pass_2021.reset_index(inplace=True)

li_1 = []
li_2 = []
li_3 = []
for idx, row in tqdm(pass_2021.iterrows()):
    for r in o_li:
        li_1.append(row["index"])
        li_2.append(r)
        li_3.append(row[r])
        
pas_21 = pd.DataFrame(list(zip(li_1, li_2, li_3)), columns=['출발지', '도착지', '통행량'])

d_li = []
for idx, row in pas_21.iterrows():
    if row['출발지'] == row['도착지']:
        d_li.append(idx)
        
pas_21.drop(d_li, inplace=True)
pas_21.reset_index(drop=True, inplace=True)
pas_21

In [ ]:
pass_2020 = pd.read_excel('./day/2020년통행량.', index_col=0)
pass_2020.reset_index(inplace=True)
pass_2020.head()

li_1 = []
li_2 = []
li_3 = []
for idx, row in tqdm(pass_2020.iterrows()):
    for r in o_li:
        li_1.append(row["index"])
        li_2.append(r)
        li_3.append(row[r])
        
pas_20 = pd.DataFrame(list(zip(li_1, li_2, li_3)), columns=['출발지', '도착지', '통행량'])
pas_20.head(10)

d_li = []
for idx, row in pas_20.iterrows():
    if row['출발지'] == row['도착지']:
        d_li.append(idx)
        
pas_20.drop(d_li, inplace=True)
pas_20.reset_index(drop=True, inplace=True)
pas_20

In [ ]:
pass_2019 = pd.read_excel('./day/2019년통행량.', index_col=0)
pass_2019.reset_index(inplace=True)
pass_2019.head()

li_1 = []
li_2 = []
li_3 = []
for idx, row in pass_2019.iterrows():
    for r in o_li:
        li_1.append(row["index"])
        li_2.append(r)
        li_3.append(row[r])
        
pas_19 = pd.DataFrame(list(zip(li_1, li_2, li_3)), columns=['출발지', '도착지', '통행량'])
pas_19.head(10)

d_li = []
for idx, row in pas_19.iterrows():
    if row['출발지'] == row['도착지']:
        d_li.append(idx)
        
pas_19.drop(d_li, inplace=True)
pas_19.reset_index(drop=True, inplace=True)
pas_19

---
# 4. 버스 데이터 전처리

## 1) 크롤링으로 수집한 주소 데이터 정리
- 서울, 경기, 인천에 위치한 정류장만 추출

In [ ]:
bus_stop = pd.read_excel('./.xlsx', index_col=0)
bus_stop = bus_stop[bus_stop['광역시/도'].isin(['서울특별시', '인천광역시', '경기도'])].reset_index(drop=True)
bus_stop

## 2) 지역 컬럼 만들기
- 통행량 데이터의 지역구분에 따른다
- 크롤링으로 구한 지하철역의 주소 데이터 사용
- 지역 컬럼에 서울시 OO구, 인천광역시 OO구/군, 경기도 OO구/군 형식으로 저장

In [ ]:
bus_stop['지역'] = bus_stop['광역시/도'] + " " + bus_stop['시/군']
bus_stop = bus_stop[bus_stop["지역"].isin(o_li)]
bus_stop

## 3) 정류장별 버스 노선

- RouteStop 과 Stop 파일을 merge하여 각 정류장에 서는 버스를 알아낸다.
- RouteStop 에서 Stop_YN = TRUE 인 데이터만 사용한다.(경유하는건 사용하지 않는다)
- 버스번호는 "RT_NM" 인데, Stop 과 RouteStop 에는 없어서,,, 다시 Route와 merge한다
- bus_stop 데이터를 이용해 서울경기인천 내 정류장들만 추출

In [ ]:
s_list = list(bus_stop['Stop_NM'].unique())

In [ ]:
line_2021 = pd.merge(Stop_bus_2021, RouteStop_bus_2021[RouteStop_bus_2021["Stop_YN"] == "Y"], on="Stop_ID", how="right")
line_2021.drop(columns=['Data_SEQ_x', 'Data_SEQ_y', 'Month_x', 'Month_y', "RT_Stop_Ord","Stop_YN","Turn_CA"], inplace=True)
line_2021 = pd.merge(line_2021, Route_bus_2021, left_on="RT_ID", right_on="RT_ID", how="left")
line_2021.drop(columns=['Month','MTC_Region', 'Data_SEQ', "RT_Desc", "RT_Group", "Region_CD"], inplace=True)
line_2021 = line_2021[line_2021['Stop_NM'].isin(s_list)]
line_21 = pd.DataFrame(pd.pivot_table(line_2021, index=['Stop_ID', 'Stop_NM', "RT_ID", "RT_NM"], values='Start', aggfunc=len)).reset_index()
line_21.drop(columns=['Start'], inplace=True)
line_21

In [ ]:
line_2020 = pd.merge(Stop_bus_2020, RouteStop_bus_2020[RouteStop_bus_2020["Stop_YN"] == "Y"], on="Stop_ID", how="right")
line_2020.drop(columns=['Data_SEQ_x', 'Data_SEQ_y', 'Month_x', 'Month_y', "RT_Stop_Ord","Stop_YN","Turn_CA"], inplace=True)
line_2020 = pd.merge(line_2020, Route_bus_2020, left_on="RT_ID", right_on="RT_ID", how="left")
line_2020.drop(columns=['Month','MTC_Region', 'Data_SEQ', "RT_Desc", "RT_Group", "Region_CD","Region_NM"], inplace=True)
line_2020 = line_2020[line_2020['Stop_NM'].isin(s_list)]
line_20 = pd.DataFrame(pd.pivot_table(line_2020, index=['Stop_ID', 'Stop_NM', "RT_ID", "RT_NM"], values='Start', aggfunc=len)).reset_index()
line_20.drop(columns=['Start'], inplace=True)
line_20

In [ ]:
line_2019 = pd.merge(Stop_bus_2019, RouteStop_bus_2019[RouteStop_bus_2019["Stop_YN"] == "Y"], on="Stop_ID", how="right")
line_2019.drop(columns=['Data_SEQ_x', 'Data_SEQ_y', 'Month_x', 'Month_y', "RT_Stop_Ord","Stop_YN","Turn_CA"], inplace=True)
line_2019 = pd.merge(line_2019, Route_bus_2019, left_on="RT_ID", right_on="RT_ID", how="left")
line_2019.drop(columns=['Month','MTC_Region', 'Data_SEQ', "RT_Desc", "RT_Group", "Region_CD","Region_NM"], inplace=True)
line_2019 = line_2019[line_2019['Stop_NM'].isin(s_list)]
line_19 = pd.DataFrame(pd.pivot_table(line_2019, index=['Stop_ID', 'Stop_NM', "RT_ID", "RT_NM"], values='Start', aggfunc=len)).reset_index()
line_19.drop(columns=['Start'], inplace=True)
line_19

## 4) 노선별 운행횟수
- 정류장별로 서는 노선의 하루 운행 회수를 정차시각이 있는 StopTime 데이터를 사용해 구했다.
- StopTime 에는 정류장 정보가 없어서, Stop_rail 데이터와 병합해서 사용했다.
```
1) Stop_rail과 StopTime 데이터를 "Stop_ID" 를 기준으로 병합
2) 필요없는 컬럼 제거
3) 피벗테이블을 사용해 노선별 운행 회수 계산
```

In [ ]:
me_2021 = pd.merge(Stop_bus_2021, StopTime_bus_2021, left_on="Stop_ID", right_on="Stop_ID", how="right")
bus_stop_count_2021 = pd.DataFrame(pd.pivot_table(data=me_2021, index=['Stop_ID', 'Stop_NM', "RT_ID"],values="ARS_ID",aggfunc=len)).reset_index()
bus_stop_count_2021.rename(columns = {'ARS_ID':'운행횟수'}, inplace = True)
bus_stop_count_2021

In [ ]:
me_2020 = pd.merge(Stop_bus_2020, StopTime_bus_2020, left_on="Stop_ID", right_on="Stop_ID", how="right")
bus_stop_count_2020 = pd.DataFrame(pd.pivot_table(data=me_2020, index=['Stop_ID', 'Stop_NM', "RT_ID"],values="ARS_ID",aggfunc=len)).reset_index()
bus_stop_count_2020.rename(columns = {'ARS_ID':'운행횟수'}, inplace = True)
bus_stop_count_2020

In [ ]:
me_2019 = pd.merge(Stop_bus_2019, StopTime_bus_2019, left_on="Stop_ID", right_on="Stop_ID", how="right")
bus_stop_count_2019 = pd.DataFrame(pd.pivot_table(data=me_2019, index=['Stop_ID', 'Stop_NM', "RT_ID"],values="ARS_ID",aggfunc=len)).reset_index()
bus_stop_count_2019.rename(columns = {'ARS_ID':'운행횟수'}, inplace = True)
bus_stop_count_2019

## 5) 데이터 합치기(연도별로)
- 위에서 만든 모든 파일을 지하철 역명을 기준으로 합쳐준다.

In [ ]:
stop_time = pd.merge(bus_stop_count_2021, line_21, on=['Stop_ID', 'Stop_NM', "RT_ID"], how='outer')
bus_2021 = pd.merge(stop_time, bus_stop, on="Stop_NM", how="left")
bus_2021.head(2)

In [ ]:
stop_time20 = pd.merge(bus_stop_count_2020, line_20, on=['Stop_ID', 'Stop_NM', "RT_ID"], how='outer')
bus_2020 = pd.merge(stop_time20, bus_stop, on="Stop_NM", how="left")
bus_2020.head(2)

In [ ]:
stop_time19 = pd.merge(bus_stop_count_2019, line_19, on=['Stop_ID', 'Stop_NM', "RT_ID"], how='outer')
bus_2019 = pd.merge(stop_time19, bus_stop, on="Stop_NM", how="left")
bus_2019.head(2)

## 6) 출발지-도착지 별 버스번호와 운행횟수 구하기
- 버스운행 컬럼 : 출발지-도착지 간 버스의 운송량
- 버스정원을 45명(광역버스 기준)으로 두고 계산

In [ ]:
tmp_bus_2021 = pd.pivot_table(data=bus_2021, index=['지역',"광역시/도", 'RT_ID', "RT_NM", "운행횟수"], values="Lati", aggfunc=len)
tmp_bus_2021.reset_index(inplace=True)
tmp_bus_2021 # Lati는 의미없음

li_1 = []
li_2 = []
li_3 = []
li_4 = []
li_5 = []
li_6 = []

for idx, row in tqdm(tmp_bus_2021.iterrows()):
    tmp_list = list(set(bus_2021[bus_2021["RT_ID"] == row['RT_ID']]['지역']))
    for i in tmp_list:
        li_1.append(row['지역'].strip())
        li_2.append(i)
        li_3.append(row['RT_NM'])
        li_4.append(row['RT_ID'])
        li_5.append(row['광역시/도'].strip())
        li_6.append(row['운행횟수'])
        
bus21 = pd.DataFrame(list(zip(li_5, li_1, li_2, li_3, li_4, li_6)), 
                   columns=["광역시/도(출발)", '출발지', '도착지', '버스번호', "RT_ID", "운행횟수"])

bus21 = bus21[~bus21['도착지'].isna()]
bus21['버스운행'] = bus21['운행횟수'] * 45
bus21

In [ ]:
bus_2020 = bus_2020[~bus_2020.duplicated(subset=["Stop_ID", "RT_ID"])]
bus_2020.head()

tmp_bus_2020 = pd.pivot_table(data=bus_2020, index=['지역',"광역시/도", 'RT_ID', "RT_NM", "운행횟수"], 
                              values="Lati", aggfunc=len)
tmp_bus_2020.reset_index(inplace=True)
tmp_bus_2020 # Lati는 의미없음

li_1 = []
li_2 = []
li_3 = []
li_4 = []
li_5 = []
li_6 = []

for idx, row in tqdm(tmp_bus_2020.iterrows()):
    tmp_list = list(set(bus_2020[bus_2020["RT_ID"] == row['RT_ID']]['지역']))
    for i in tmp_list:
        li_1.append(row['지역'])
        li_2.append(i)
        li_3.append(row['RT_NM'])
        li_4.append(row['RT_ID'])
        li_5.append(row['광역시/도'])
        li_6.append(row['운행횟수'])
        
bus20 = pd.DataFrame(list(zip(li_5, li_1, li_2, li_3, li_4, li_6)), 
                   columns=["광역시/도(출발)", '출발지', '도착지', '버스번호', "RT_ID", "운행횟수"])

bus20 = bus20[~bus20['도착지'].isna()]
bus20['버스운행'] = bus20['운행횟수'] * 45

bus20

In [ ]:
bus_2019 = bus_2019[~bus_2019.duplicated(subset=["Stop_ID", "RT_ID"])]
bus_2019.head()

tmp_bus_2019 = pd.pivot_table(data=bus_2019, index=['지역',"광역시/도", 'RT_ID', "RT_NM", "운행횟수"], 
                              values="Lati", aggfunc=len)
tmp_bus_2019.reset_index(inplace=True)
tmp_bus_2019 # Lati는 의미없음

li_1 = []
li_2 = []
li_3 = []
li_4 = []
li_5 = []
li_6 = []

for idx, row in tqdm(tmp_bus_2019.iterrows()):
    tmp_list = list(set(bus_2019[bus_2019["RT_ID"] == row['RT_ID']]['지역']))
    for i in tmp_list:
        li_1.append(row['지역'])
        li_2.append(i)
        li_3.append(row['RT_NM'])
        li_4.append(row['RT_ID'])
        li_5.append(row['광역시/도'])
        li_6.append(row['운행횟수'])
        
bus19 = pd.DataFrame(list(zip(li_5, li_1, li_2, li_3, li_4, li_6)), 
                   columns=["광역시/도(출발)", '출발지', '도착지', '버스번호', "RT_ID", "운행횟수"])
bus19 = bus19[~bus19['도착지'].isna()]
bus19['버스운행'] = bus19['운행횟수'] * 45

bus19

---
# 5. 전철 데이터 전처리

## 1) 크롤링으로 수집한 주소 데이터 정리 & 지역컬럼 만들기
- 통행량 데이터의 지역구분에 따른다.
- 크롤링으로 구한 지하철역의 주소 데이터 사용
- 지역 컬럼에 서울시 OO구, 인천광역시 OO구/군, 경기도 OO구/군 형식으로 저장

In [ ]:
rail_stop = pd.read_csv('./.csv', index_col=0)
rail_stop['지역'] = rail_stop['광역시/도'] + " " + rail_stop['시/군']
rail_stop.sort_values(by="지역")
rail_stop = rail_stop[rail_stop["지역"].isin(o_li)]
rail_stop.head()

## 2) 노선별 하루 운행 회수 구하기
- 정류장별로 서는 노선의 하루 운행 회수를 정차시각이 있는 StopTime 데이터를 사용해 구했다.
- StopTime 에는 정류장 정보가 없어서, Stop_rail 데이터와 병합해서 사용했다. 
```
1) Stop_rail과 StopTime 데이터를 "Stop_ID" 를 기준으로 병합
2) 필요없는 컬럼 제거
3) 피벗테이블을 사용해 노선별 운행 회수 계산
```

In [ ]:
me_2021 = pd.merge(Stop_rail_2021, StopTime_rail_2021, on="Stop_ID", how="right")
me_2021.drop(columns=['Data_SEQ_x', 'Data_SEQ_y', 'Month_x', 'Month_y', 'Gate'], inplace=True) 

rail_stop_count_2021 = pd.DataFrame(pd.pivot_table(data=me_2021, index=['Stop_ID', 'Stop_NM', "RT_ID"],values="Arr_Time",aggfunc=len)).reset_index()
rail_stop_count_2021.head()

In [ ]:
me_2020 = pd.merge(Stop_rail_2020, StopTime_rail_2020, on="Stop_ID", how="right")
me_2020.drop(columns=['Data_SEQ_x', 'Data_SEQ_y', 'Month_x', 'Month_y', 'Gate'], inplace=True) 

rail_stop_count_2020 = pd.DataFrame(pd.pivot_table(data=me_2020, index=['Stop_ID', 'Stop_NM', "RT_ID"],values="Arr_Time",aggfunc=len)).reset_index()
rail_stop_count_2020.head()

In [ ]:
me_2019 = pd.merge(Stop_rail_2019, StopTime_rail_2019, on="Stop_ID", how="right")
me_2019.drop(columns=['Data_SEQ_x', 'Data_SEQ_y', 'Month_x', 'Month_y', 'Gate'], inplace=True) 

rail_stop_count_2019 = pd.DataFrame(pd.pivot_table(data=me_2019, index=['Stop_ID', 'Stop_NM', "RT_ID"],values="Arr_Time",aggfunc=len)).reset_index()
rail_stop_count_2019.head()

## 3) 역별 호선, 칸 개수
- 모든 정류장의 량수 필요
- Stop 에 정류장 정보가, Route 에 호선 별 량수 정보가 있음
- 둘을 바로 병합할수 없어서 중간에 RouteStop 을 사용함

```
1) Stop_rail 파일과 RouteStop 파일을 "Stop_ID" 를 기준으로 병합
2) 필요없는 컬럼 제거
3) 다시 Route_rail 파일과 "RT_ID" 를 기준으로 병합 (Route_rail 파일에 호선별 칸수 데이터가 있음)
```

In [ ]:
line_2021 = pd.merge(Stop_rail_2021, RouteStop_rail_2021, on="Stop_ID", how="left")
line_2021.drop(columns=['Data_SEQ_x', 'Data_SEQ_y', 'Month_x', 'Month_y',"Gate"], inplace=True)
line_2021 = pd.merge(line_2021, Route_rail_2021, on="RT_ID", how="left")
line_2021.fillna(0, inplace=True)
line_2021= line_2021[line_2021['객실수'] != 0]
line_2021.head()

In [ ]:
line_2020 = pd.merge(Stop_rail_2020, RouteStop_rail_2020, on="Stop_ID", how="left")
line_2020.drop(columns=['Data_SEQ_x', 'Data_SEQ_y', 'Month_x', 'Month_y',"Gate"], inplace=True)
line_2020 = pd.merge(line_2020, Route_rail_2020, on="RT_ID", how="left")
line_2020.fillna(0, inplace=True)
line_2020= line_2020[line_2020['객실수'] != 0]
line_2020.head()

In [ ]:
line_2019 = pd.merge(Stop_rail_2019, RouteStop_rail_2019, on="Stop_ID", how="left")
line_2019.drop(columns=['Data_SEQ_x', 'Data_SEQ_y', 'Month_x', 'Month_y',"Gate"], inplace=True)
line_2019 = pd.merge(line_2019, Route_rail_2019, on="RT_ID", how="left")
line_2019.fillna(0, inplace=True)
line_2019= line_2019[line_2019['객실수'] != 0]
line_2019.head()


## 4) 데이터 합치기(연도별로)
- 위에서 만든 모든 파일을 지하철 역명을 기준으으로 합쳐준다.
- 파일들을 함수에 입력하면 된다.

In [ ]:
tmp21 = pd.merge(rail_stop_count_2021, line_2021,on=['RT_ID', "Stop_NM"], how='outer')
tmp21.drop(columns=['Stop_ID_x', "Data_SEQ", "Month", "RT_Stop_Ord"], inplace=True)
rail_2021 = pd.merge(tmp21, rail_stop, on='Stop_NM', how='left')

In [ ]:
tmp20 = pd.merge(rail_stop_count_2020, line_2020,on=['RT_ID', "Stop_NM"], how='outer')
tmp20.drop(columns=['Stop_ID_x', "Data_SEQ", "Month", "RT_Stop_Ord"], inplace=True)
rail_2020 = pd.merge(tmp20, rail_stop, on='Stop_NM', how='left')

In [ ]:
tmp19 = pd.merge(rail_stop_count_2019, line_2019,on=['RT_ID', "Stop_NM"], how='outer')
tmp19.drop(columns=['Stop_ID_x', "Data_SEQ", "Month", "RT_Stop_Ord"], inplace=True)
rail_2019 = pd.merge(tmp20, rail_stop, on='Stop_NM', how='left')

## 5) 출발지-도착지 별 노선명과 운행 회수
- 전철운행 컬럼 : 출발지-도착지 간 전철의 운송량
- 1칸 당 160명을 정원으로 두고 계산

In [ ]:
rail_2021.fillna(0, inplace=True) # 이거 다시 생각해보기
rail_2021 = rail_2021[rail_2021['객실수'] != 0]
rail_2021.reset_index(drop=True, inplace=True)

tmp_rail_2021 =pd.DataFrame(pd.pivot_table(rail_2021, index=['지역', "광역시/도", 'RT_NM', '객실수'], values="운행횟수", aggfunc=np.sum))
tmp_rail_2021.reset_index(inplace=True)

li_1 = []
li_2 = []
li_3 = []
li_4 = []
li_5 = []
li_6 = []
li_7 = []
li_8 = []
li_9 = []

for idx, row in tqdm(tmp_rail_2021.iterrows()):
    tmp_list = list(set(rail_2021[rail_2021["RT_NM"] == row['RT_NM']]['지역']))
    for i in tmp_list:
        li_1.append(row['광역시/도'].strip())
        #li_2.append(row['시/군'])
        #li_3.append(row['구'])
       # li_4.append(row['동/읍/면'])
        li_5.append(row['지역'].strip())
        li_6.append(i)
        li_7.append(row['RT_NM'])
        li_8.append(row['객실수'])
        li_9.append(row['운행횟수'])
        
rail21 = pd.DataFrame(list(zip(li_1,li_5,li_6,li_7, li_8, li_9)),
                    columns=["광역시도(출발)", '출발지', '도착지','RT_NM', '객실수', '운행횟수'])
rail21['전철운행'] = rail21['객실수'] * rail21['운행횟수'] * 160
rail21 = rail21[~rail21['도착지'].isna()]
rail21

In [ ]:
rail_2020.fillna(0, inplace=True) # 이거 다시 생각해보기
rail_2020 = rail_2020[rail_2020['객실수'] != 0]
rail_2020.reset_index(drop=True, inplace=True)
rail_2020.info()

tmp_rail_2020 =pd.DataFrame(pd.pivot_table(rail_2020, index=['지역', "광역시/도", 'RT_NM', '객실수'], 
                                           values="운행횟수", aggfunc=np.sum))
tmp_rail_2020.reset_index(inplace=True)
tmp_rail_2020.head()



li_1 = []
li_5 = []
li_6 = []
li_7 = []
li_8 = []
li_9 = []

for idx, row in tmp_rail_2020.iterrows():
    tmp_list = list(set(rail_2020[rail_2020["RT_NM"] == row['RT_NM']]['지역']))
    for i in tmp_list:
        li_1.append(row['광역시/도'])
        li_5.append(row['지역'])
        li_6.append(i)
        li_7.append(row['RT_NM'])
        li_8.append(row['객실수'])
        li_9.append(row['운행횟수'])
        
        
rail20 = pd.DataFrame(list(zip(li_1,li_5,li_6,li_7, li_8, li_9)),
                    columns=["광역시도(출발)", '출발지', '도착지','RT_NM', '객실수', '운행횟수'])
rail20['전철운행'] = rail20['객실수'] * rail20['운행횟수'] * 160
rail20 = rail20[~rail20['도착지'].isna()]
rail20

In [ ]:

rail_2019.fillna(0, inplace=True) # 이거 다시 생각해보기

rail_2019 = rail_2019[rail_2019['객실수'] != 0]  # 이거 다시 생각해보기
rail_2019.reset_index(drop=True, inplace=True)
rail_2019.info()

tmp_rail_2019 =pd.DataFrame(pd.pivot_table(rail_2019, index=['지역', "광역시/도", 'RT_NM', '객실수'], 
                                           values="운행횟수", aggfunc=np.sum))
tmp_rail_2019.reset_index(inplace=True)
tmp_rail_2019.head()



li_1 = []
li_5 = []
li_6 = []
li_7 = []
li_8 = []
li_9 = []

for idx, row in tmp_rail_2019.iterrows():
    tmp_list = list(set(rail_2019[rail_2019["RT_NM"] == row['RT_NM']]['지역']))
    for i in tmp_list:
        li_1.append(row['광역시/도'])
        li_5.append(row['지역'])
        li_6.append(i)
        li_7.append(row['RT_NM'])
        li_8.append(row['객실수'])
        li_9.append(row['운행횟수'])
        
        
rail19 = pd.DataFrame(list(zip(li_1,li_5,li_6,li_7, li_8, li_9)),
                    columns=["광역시도(출발)", '출발지', '도착지','RT_NM', '객실수', '운행횟수'])
rail19['전철운행'] = rail['객실수'] * rail['운행횟수'] * 160
rail19 = rail19[~rail19['도착지'].isna()]


rail19.to_excel('./aa/2019전철table.xlsx', encoding='utf-8')

---
# 6. 통행량, 버스, 전철 데이터 하나로 합치기
- 출발지 - 도착지 간 버스/전철의 수송량, 통행량 데이터로 만들기
- 교통적정 컬럼: 통행량 - 대중교통 수송량
- 교통적정 > 0 : 대중교통 노선 부족
- 교통적정 < 0 : 대중교통 노선 충분

In [ ]:
pas_21["출발지"] = pas_21["출발지"].str.strip()
pas_21["도착지"] = pas_21["도착지"].str.strip()

rail_sum["출발지"] = rail_sum["출발지"].str.strip()
rail_sum["도착지"] = rail_sum["도착지"].str.strip()

bus_sum["출발지"] = bus_sum["출발지"].str.strip()
bus_sum["도착지"] = bus_sum["도착지"].str.strip()

tmp21 = pd.merge(bus_sum, rail_sum, on=["출발지", "도착지"], how='outer')
final_2021 = pd.merge(pas_21, tmp21, on=["출발지", "도착지"], how='left')

final_2021.fillna(0, inplace=True)
final_2021['교통적정'] = final_2021['통행량'] - (final_2021['전철운행'] + final_2021['버스운행'])
final_2021['연도'] = 2021
final_2021.to_excel('./day/final_2021.xlsx', encoding='utf-8')

In [ ]:
rail_sum20 = pd.DataFrame(pd.pivot_table(data=rail20, index=['출발지', '도착지'], values='전철운행', aggfunc=np.sum))
rail_sum20.reset_index(inplace=True)

bus_sum20 = pd.DataFrame(pd.pivot_table(data=bus20, index=['출발지', '도착지'], values='버스운행', aggfunc=np.sum))
bus_sum20.reset_index(inplace=True)

pas_20["출발지"] = pas_20["출발지"].str.strip()
pas_20["도착지"] = pas_20["도착지"].str.strip()

rail_sum20["출발지"] = rail_sum20["출발지"].str.strip()
rail_sum20["도착지"] = rail_sum20["도착지"].str.strip()

bus_sum20["출발지"] = bus_sum20["출발지"].str.strip()
bus_sum20["도착지"] = bus_sum20["도착지"].str.strip()

tmp20 = pd.merge(pas_20, rail_sum20, on=["출발지", "도착지"], how='left')
final_2020 = pd.merge(tmp20, bus_sum20, on=["출발지", "도착지"], how='left')
final_2020.fillna(0, inplace=True)
final_2020['교통적정'] = final_2020['통행량'] - (final_2020['전철운행'] + final_2020['버스운행'])
final_2020['연도'] = 2020

final_2020.to_excel('./day/final_2020.xlsx', encoding='utf-8')

In [ ]:
rail_sum19 = pd.DataFrame(pd.pivot_table(data=rail19, index=['출발지', '도착지'], values='전철운행', aggfunc=np.sum))
rail_sum19.reset_index(inplace=True)

bus_sum19 = pd.DataFrame(pd.pivot_table(data=bus19, index=['출발지', '도착지'], values='버스운행', aggfunc=np.sum))
bus_sum19.reset_index(inplace=True)


pas_19["출발지"] = pas_19["출발지"].str.strip()
pas_19["도착지"] = pas_19["도착지"].str.strip()

rail_sum19["출발지"] = rail_sum19["출발지"].str.strip()
rail_sum19["도착지"] = rail_sum19["도착지"].str.strip()

bus_sum19["출발지"] = bus_sum19["출발지"].str.strip()
bus_sum19["도착지"] = bus_sum19["도착지"].str.strip()


tmp19 = pd.merge(pas_19, rail_sum19, on=["출발지", "도착지"], how='left')
final_2019 = pd.merge(tmp19, bus_sum19, on=["출발지", "도착지"], how='left')
final_2019.fillna(0, inplace=True)
final_2019['교통적정'] = final_2019['통행량'] - (final_2019['전철운행'] + final_2019['버스운행'])
final_2019['연도'] = 2019

final_2019.to_excel('./day/final_2019.xlsx', encoding='utf-8')

In [ ]:
final_total = pd.concat([final_2019, final_2020, final_2021])
final_total.to_excel('./day/final_total.xlsx', encoding='utf-8')